# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan and Carly Hu, for Cornell Data Journal_

In [1]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# update access keys here for Spotify API here :) 
client_id = 'f7661e34c0924317a8b928c0f977e6a7'
client_secret = 'd068ec3f0a98490e9c32be820916e934'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

<h3> Look at your own stats! </h3>

Feel free to put in your own spotify username in `selected_user` below, and if you run every cell, by the end of the notebook you will be able to see the information & statistics for __all of the songs that are on your public playlists__ (with duplicates removed). Have fun!

In [5]:
# Big Data, Machine Learning...
# https://www.youtube.com/watch?v=bqqCTC9nQDY

# let's aggregate an entire user's public library of songs
playlists = []
LIMIT = 20
selected_user = 'carlatude'

query = sp.user_playlists(selected_user, limit=LIMIT)

n = 0

while len(query['items']) != 0:
    n += LIMIT
    for i in query['items']:
        playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
    if LIMIT == len(query['items']):
        query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
    else:
        break
        
# there's definitely a cleaner way to do this that we'll figure out
# im definitely lazy

In [6]:
playlists

[{'title': 'spring 23', 'id': '0AberywlM7KTsCBRAs1Hs0', 'length': 46},
 {'title': 'bouncing', 'id': '7lspAgfh11vTTTpm5G1wkf', 'length': 12},
 {'title': 'kenny', 'id': '4jX7u824J8LFH7iYP6d44G', 'length': 9},
 {'title': 'walking energetically',
  'id': '09EmE7ZmePNp8qtvzePdWE',
  'length': 16},
 {'title': 'instrumental', 'id': '42DYI4rZtxyySmX3sjAoBy', 'length': 40},
 {'title': 'pretty', 'id': '2EXVB9JsrIPINY25rJ7Bth', 'length': 65},
 {'title': 'comfort', 'id': '7645opI5xNijOpeaQ6wOJK', 'length': 41},
 {'title': 'screaming crying', 'id': '2C1QFkfhQDDw0jYm5Tr9pE', 'length': 34},
 {'title': 'slay pt 2', 'id': '2U9jxw1KdLtCATpYQZC0qv', 'length': 47},
 {'title': 'taylor (sad)', 'id': '5WCNl7XT1KuRN2nLyhtmBH', 'length': 51},
 {'title': 'agh!!!!', 'id': '0zefXRE0Pr0W72A5966PLn', 'length': 27},
 {'title': 'wake up', 'id': '0DN6Ihs7y7XarkOgXpMkau', 'length': 19},
 {'title': 'groovy', 'id': '7oKjtKN4PQJ1ZA6n5N9qfN', 'length': 118},
 {'title': 'bedroom pop', 'id': '6QGojHMaB0ktJ32vpsxRZk', 'length

In [7]:
# number of public playlists
print(len(playlists))

29


In [8]:
# making sure we can get all songs in a playlist rather than just the first 100
# https://stackoverflow.com/questions/55690063/is-there-a-method-to-retrieve-all-tracks-from-a-playlist-using-spotipy 

def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results
        

In [ ]:
# now, let's go through each playlist
# this will basically be the same as before

songs = []

for p in playlists:
    
    results = sp.playlist(p['id'])
    
    extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100
    
    for r in extended_results:
        songs.append(r['track']['id']) 

print(len(songs)) # this is the total number of songs on your public playlists

In [ ]:
# helper function
def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = (sp.tracks(songs[len(results):len(results) + 50]))['tracks']
        results.extend(response)
    return results

In [ ]:
# this cell is creating a dataframe of your song information: artist, album, popularity, etc.

songs_meta = get_tracks_full(songs)

song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
             'explicit': [], 'popularity': []}
    
for i in songs_meta:
    # append id to id column
    song_info['id'].append(i['id'])
    
    # append song title to title column
    song_info['title'].append(i['name'])
    
    # append artists to artist column
    s = ', '
    artists = s.join([name['name'] for name \
                     in i['artists']])
    song_info['artist'].append(artists)
    
    # append album title to album column
    song_info['album'].append(i['album']['name'])
    
    # append explicit info to explicit column
    song_info['explicit'].append(i['explicit'])
    
    # append popularity info to popularity column
    song_info['popularity'].append(i['popularity'])
    
# convert song_info dictionary -> Pandas DataFrame
song_info_df = pd.DataFrame.from_dict(song_info)

song_info_df = song_info_df.drop_duplicates(subset=['id'])
song_info_df = song_info_df.dropna()
print(song_info_df.columns)
print(song_info_df.shape)

In [ ]:
#helper function 

def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        results.extend(response)
    return results

In [ ]:
# this cell is creating a dataframe of your song features: danceability, energy, loudness, etc

features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))
features_df = features_df.drop_duplicates(subset=['id'])
features_df = features_df.dropna()
print(features_df.columns)
print(features_df.shape)

In [ ]:
# combining the two cleaned dataframes....
df = song_info_df.merge(features_df)
print(df.shape)
df